In [1]:
import pandas as pd
from pyvi import ViUtils
import re

In [2]:
ddv_df = pd.read_csv('../data/didongviet.csv')
for column in ddv_df:
    print(column)

Tên sản phẩm
Gía
Bluetooth 
chip
Màn hình
Công nghệ màn hình
pin
sim
trọng lượng
RAM
Độ phân giải
ROM


In [3]:
drop_columns = ['Bluetooth ', 'trọng lượng', 'Độ phân giải']

for column in drop_columns:
    del ddv_df[column]

ddv_df.head()

,Tên sản phẩm,Gía,chip,Màn hình,Công nghệ màn hình,pin,sim,RAM,ROM
0,iPhone 13 256GB Chính Hãng (VN/A),21.990.000 ₫,Apple A15 Bionic,"6.1"" - Tần số quét 60 Hz",OLED,NaN,1 Nano SIM & 1 eSIM,4 GB,256 GB
1,iPhone 13 Pro 128GB Chính Hãng (VN/A),24.990.000 ₫,Apple A15 Bionic,"6.1"" - Tần số quét 120 Hz",Super Retina XDR - ProMotion,NaN,1 Nano SIM & 1 eSIM,6 GB,128 GB
2,iPhone 13 Pro 256GB Chính Hãng (VN/A),27.990.000 ₫,Apple A15 Bionic,"6.1"" - Tần số quét 120 Hz",Super Retina XDR - ProMotion,NaN,1 Nano SIM & 1 eSIM,6 GB,256 GB
3,iPhone 13 Pro 512GB Chính Hãng (VN/A),32.990.000 ₫,Apple A15 Bionic,"6.1"" - Tần số quét 120 Hz",Super Retina XDR - ProMotion,NaN,1 Nano SIM & 1 eSIM,6 GB,512 GB
4,iPhone 13 Pro Max 128GB Chính Hãng (VN/A),26.990.000 ₫,Apple A15 Bionic,"6.7"" - Tần số quét 120Hz",Super Retina XDR - ProMotion,4325 mAh,1 Nano SIM & 1 eSIM,6 GB,128 GB


In [4]:
print(len(ddv_df))

859


# Xử lý tên cột

In [5]:
ddv_df = ddv_df.rename(columns={"Tên sản phẩm": "ten", "Gía": "gia", "chip": "cpu", "Màn hình": "kich_thuoc_man_hinh", "ROM": "bo_nho_trong"})

for column in ddv_df.columns:
    ddv_df = ddv_df.rename(columns={column: ViUtils.remove_accents(column).decode("utf-8").lower().strip().replace(' ', '_')})

# Xử lý trường tên

In [6]:
ddv_df = ddv_df[ddv_df['ten'].notna()]
ddv_df['ten'] = ddv_df['ten'].apply(lambda s: str(s).replace('Chính Hãng', '').replace('(VN/A)', '').replace('I', '').replace('Điện thoại', '').strip())
ddv_df['ten'] = ddv_df['ten'].apply(lambda s: ViUtils.remove_accents(s).decode("utf-8"))

# Xử lý trường giá

In [7]:
ddv_df = ddv_df[ddv_df['gia'].notna()]
ddv_df['gia'] = ddv_df['gia'].apply(lambda s: str(s).replace('₫', '').replace('.', '').replace('*', '').strip())
ddv_df['gia'] = ddv_df['gia'].apply(lambda s: int(s))

# Xử lý trường kích thước màn hình

In [8]:
def process_screen_size(s):
    try:
        size = re.findall(r"[-+]?(?:\d*\.\d+|\d+)", s)
        res = 0
        for si in size:
            si = float(si)
            if si < 20:
                res += si
        return res
    except:
        return 0

ddv_df['kich_thuoc_man_hinh'] = ddv_df['kich_thuoc_man_hinh'].apply(process_screen_size)

# Xử lý trường bộ nhớ trong

In [9]:
def process_rom(s):
    try:
        if 'GB' in s:
            return float(s[:-2].strip())
        elif 'MB' in s:
            return float(s[:-2].strip())/1024
        else:
            return float(s[:-2].strip())*1024
    except:
        return 0

ddv_df['bo_nho_trong'] = ddv_df['bo_nho_trong'].apply(process_rom)

# Xử lý trường RAM

In [10]:
def process_ram(s):
    try:
        if 'GB' in s:
            return float(s[:-2].strip())
        elif 'MB' in s:
            return float(s[:-2].strip())/1024
        else:
            return float(s[:-2].strip())*1024
    except:
        return 0

ddv_df['ram'] = ddv_df['ram'].apply(process_ram)

In [11]:
for column in ddv_df.columns:
    print(column)

ten
gia
cpu
kich_thuoc_man_hinh
cong_nghe_man_hinh
pin
sim
ram
bo_nho_trong


In [12]:
ddv_df = ddv_df.drop_duplicates()

In [13]:
ddv_df.to_csv('../clean_data/didongviet.csv', index=False)